## Lecture 5: Matrix Norms, SVD, Compression Errors

In [ ]:
import numpy as np
import os

import matplotlib.pyplot as plt
from matplotlib import rc

plt.rcParams['xtick.labelsize']=20      # change the tick label size for x axis
plt.rcParams['ytick.labelsize']=20      # change the tick label size for x axis
plt.rcParams['axes.linewidth']=1        # change the line width of the axis
plt.rcParams['xtick.major.width'] = 3   # change the tick line width of x axis
plt.rcParams['ytick.major.width'] = 3   # change the tick line width of y axis
rc('text', usetex=False)                # disable LaTeX rendering in plots
rc('font',**{'family':'DejaVu Sans'})   # set the font of the plot to be DejaVu Sans

### 1. SVD on dog.jpg

Let's first review what we did in Lecture 2 and load the dog.jpg file into Python

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/ME491"
image_path = os.path.join(path, "image/dog.jpg")

Mounted at /content/drive


In [ ]:
from matplotlib.image import imread

A = imread(image_path)

Now we have read in the picture, let's construct matrix X for SVD by converting the image to grayscale.

In [ ]:
X = np.mean(A, -1) # Convert RGB to grayscale
plt.imshow(X, cmap="Greys_r")
plt.axis('off') # We don't need axis here so let's remove them

Recall the equation for SVD, we have
$$ X = U\Sigma V^*$$.

Let's now use the numpy build-in SVD function to find $U, \Sigma$ and $V^*$.
Documentation: https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html

Recall in lecture 3 we talked about full SVD and economy SVD, `numpy.linalg.svd` can do both, let's look at the results from both of them.

In [ ]:
# Take the Full SVD
U_full, S_full, VT_full =
# Take the economy SVD
U_econ, S_econ, VT_econ =
# Let's look the shape of all matrices
print("Dimensions of U, S, VT for Full SVD are: \n",
      np.shape(U_full), np.shape(S_full), np.shape(VT_full))
print("Dimensions of U, S, VT for Economy SVD are: \n",
      np.shape(U_econ), np.shape(S_econ), np.shape(VT_econ))

Do they both return X?

In [ ]:
# For full SVD, we need to generate a matrix of nxm
# to do the matrix product
S_full_mat = np.zeros(X.shape)
np.fill_diagonal(S_full_mat, S_full)
X_full = U_full @ S_full_mat @ VT_full
X_econ = U_econ @ np.diag(S_econ) @ VT_econ

In [ ]:
print("Full SVD returns X: ", np.array_equal(X_full, X))
print("Economy SVD returns X: ", np.array_equal(X_econ, X))
print("Full and economy SVD returns the same: ", np.array_equal(X_full, X_econ))
print("Full SVD returns X: ", np.allclose(X_full, X, atol=1e-8))
print("Economy SVD returns X: ", np.allclose(X_econ, X, atol=1e-8))
print("Full and economy SVD returns the same: ", np.allclose(X_full, X_econ, atol=1e-15))

### 2. Let's compress dog.jpg

First, let's find the rank of $X$.  We know it at most has a rank of 1500, but let's check if all singular values are non-zero.

From now on, we will use the results we got from Economy SVD.

In [ ]:
# let's assign new names so we can type less
U, S, VT = U_econ, S_econ, VT_econ

# we can use np.where to find the indices for certain conditions
idx =
print(len(idx[0]))

There is no zero singular values, so $\text{rank}(S)=1500$. Any rank $r$ matrix for $r<1500$ will be an approximation of $X$.

Let's now apply approximation for the following $r$:
$$ r = [1, 5, 20, 100, 100] $$

In [ ]:
# Let's make 5 subplots in a row
fig, axs = plt.subplots(1, 5)
j = 0
S_diag = np.diag(S)
for r in (1000, 100, 20, 5, 1):
    # Construct approximate image
    # approximate image with first r SVD modes
    # U_approx: nxr
    # S_approx: rxr
    # VT_approx: rxm
    Xapprox =
    img = axs[j].imshow(Xapprox, cmap='Greys_r')
    axs[j].axis('off')
    axs[j].set_title('r = ' + str(r))
    j += 1

### 3. Error Calculations

In lecture 3, we learned that the error of the SVD approximation is given by the Frobenius norm.  Because of the properties of the matrices generated by SVD, the error can be simplied to
$$||X-\tilde{X}||_F^2 = \sum_{k=r+1}^m\sigma_k^2 $$

In this part, let's use `numpy` to find the Frobenius norm of a matrix, and then compare the results with the summation.

Note: `numpy.linalg.norm` provides many options to compute different kinds of norms and the default `norm` is the Frobenius norm. For more info, check out the documentation page: https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html

In [ ]:
S_sq = S**2
S_sq_sum = np.sum(S_sq)
for r in (1000, 100, 20, 5, 1):
    Xapprox = U[:,:r] @ S_diag[:r,:r] @ VT[:r,:]
    X_diff = X - Xapprox
    F_norm_sq =
    print("The Frobenius norm is: ", F_norm_sq)
    print("The sum of remaining singular values is: ", )

You probably noticed that all these numbers are quite big, so let's now find the relative error.

In [ ]:
X_norm_sq = np.linalg.norm(X)**2
for r in (1000, 100, 20, 5, 1):
    Xapprox = U[:,:r] @ S_diag[:r,:r] @ VT[:r,:]
    X_diff = X - Xapprox
    F_norm_sq =
    print("The relative error from direct norm computation is: ",  "{:%}".format())
    print("The relative error from summing singular values is: ",  "{:%}".format())

### 4. Determine rank $r$ given an error estimation

Many times, we want to approximate the matrix to a specific value, let's see we want to keep the error to be below $10\%$, and then compute what rank $r$ we would need.

Now let's use `numpy.cumsum` and `numpy.where` to find the $r$ value necessary.



In [ ]:
# Let's find the r necessary for error 0.01, 0.02, 0.05, and 0.1
S_sq_cumsum = np.cumsum(S_sq)
for error in [0.01, 0.02, 0.05, 0.1]:
  all_accuracy =
  idx =
  r =
  print("Rank r for error", error, "is :", r)

### 5. Connection to Eigenvalue Decomposition

We can relate the SVD to an eigenvalue problem by doing the following transformation
$$XX^* = U\begin{bmatrix}
    \hat{\Sigma}  \\
    0
\end{bmatrix} V^*V\begin{bmatrix}
    \hat{\Sigma}  & 0
\end{bmatrix}U^* = U\begin{bmatrix}
    \hat{\Sigma}^2 & 0  \\
    0 & 0
\end{bmatrix}U^*$$
$$X^*X = V\begin{bmatrix}
    \hat{\Sigma}  & 0
\end{bmatrix}U^*U\begin{bmatrix}
    \hat{\Sigma}  \\
    0
\end{bmatrix} V^* = V\hat{\Sigma}^2V^*$$

Recall that $U$ and $V$ are unitary matrices, so $U,\Sigma,V$ are solutions to the following eigenvalue problems:

$$XX^*U=U\begin{bmatrix}
    \hat{\Sigma}^2 & 0  \\
    0 & 0
\end{bmatrix}$$
$$X^*XV=V\hat{\Sigma}^2$$

Now let's test them out

First, let's compute $XX^*$ and $X^*X$

In [ ]:
# to use the build-in function to find X^*, we need to convert
# X from an np.ndarray to a matrix
X_mat = np.matrix(X)
# we convert back to arrays to utilize more generalized functionalities
Xh = np.array(X_mat.getH())
XXh = np.matmul(X, Xh)
XhX = np.matmul(Xh, X)

Let's plot both products to see how they look

In [ ]:
# Let's make 5 subplots in a row
print("The shape of XX^* is: ", XXh.shape)
print("The shape of X^*X is: ", XhX.shape)
fig, axs = plt.subplots(1, 2)
img = axs[0].imshow(XXh, cmap='Greys_r')
axs[0].axis('off')
axs[0].set_title('$XX^*$')
img = axs[1].imshow(XhX, cmap='Greys_r')
axs[1].axis('off')
axs[1].set_title('$X^*X$')

Let's now find the eigenvectors and eigenvalues for both square matrices

In [ ]:
XXh_value, XXh_vector = np.linalg.eig(XXh)
XhX_value, XhX_vector = np.linalg.eig(XhX)

Let's compare the eigenvalues to $\Sigma^2$

In [ ]:
plt.semilogy(np.real(XXh_value), linewidth = 2, label = "Eigenvalues for $XX^*$")
plt.semilogy(np.real(XhX_value), linewidth = 2, label = "Eigenvalues for $X^*X$")
plt.semilogy(S_sq, linewidth = 2, label = "Singular Value Squares")
plt.legend(frameon = False)

### 6. In-class exercise

1. generate a matrix of $100\times50$ with random values of your choosing
2. plot the matrix using `imshow`
3. perform economy SVD for the matrix
4. confirm the rank of the matrix
5. choose 3 different ranks and apply matrix approximation
6. plot the 3 approximated matrices
